In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import tensorflow as tf

from bayesflow import benchmarks
from bayesflow.amortizers import AmortizedPosterior, AmortizedLikelihood, AmortizedPosteriorLikelihood
from bayesflow.networks import InvertibleNetwork
from bayesflow.trainers import Trainer
from bayesflow.diagnostics import plot_losses

from custom_plots import plot_sbc_ecdf

In [ ]:
# Comment out, if you want tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [3]:
# Parameters for publication-ready main text plot (Figure 3)
plt.rcParams.update({
    "axes.labelsize" : 24,
    "xtick.labelsize" : 16,
    "ytick.labelsize" : 16,
    "legend.fontsize": 24,
    "text.usetex": False,
    "font.family": "serif",
    'text.latex.preamble' : r'\usepackage{{amsmath}}'
})

# Benchmark: Gaussian Mixture

In [4]:
benchmark_name = 'gaussian_mixture'
benchmark = benchmarks.Benchmark(benchmark_name)

In [5]:
def custom_configure(input_dict, preconfigured=False):
    """Scales down default prior range from [-10, 10] to [-1, 1].
    Not necessary, but a good practice when working with NNs.
    """

    if not preconfigured:
        input_dict = benchmark.configurator(input_dict)
    input_dict['posterior_inputs']['parameters'] /= 10
    input_dict['likelihood_inputs']['conditions'] /= 10
    return input_dict

In [6]:
# Coupling settings
settings = {
    'dense_args': dict(units=64, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    'dropout_prob': 0.05,
    'num_dense': 1
}

# Inference networks
inference_net_p = InvertibleNetwork(
    num_params=2, 
    num_coupling_layers=6,
    permutation='learnable',
    coupling_design='spline',
    coupling_settings=settings,
)

inference_net_l = InvertibleNetwork(
    num_params=2, 
    num_coupling_layers=6,
    coupling_design='spline',
    coupling_settings=settings,
)

# Amortizers
amortized_posterior = AmortizedPosterior(
    inference_net=inference_net_p,
)

amortized_likelihood = AmortizedLikelihood(
    surrogate_net=inference_net_l
)

amortizer = AmortizedPosteriorLikelihood(
    amortized_posterior, 
    amortized_likelihood
)

In [7]:
trainer = Trainer(
    amortizer=amortizer,
    generative_model=benchmark.generative_model,
    configurator=custom_configure,
    checkpoint_path=f'checkpoints/{benchmark_name}',
    memory=False,
    max_to_keep=1
)

# Training
Uncomment for training. Wall time on GPU approx. 41min 52s

In [8]:
# EPOCHS = 150
# SIMULATION_BUDGET = 10000
# N_VAL = 300
# BATCH_SIZE = 64

# train_data = trainer.generative_model(SIMULATION_BUDGET)

In [9]:
# %%time
# h = trainer.train_offline(train_data, EPOCHS, BATCH_SIZE, validation_sims=N_VAL)

# Validation

## Loss Curves

In [10]:
# # Use loaded history, since reference 'h' will only exist after training
h = trainer.loss_history.get_plottable()
f = plot_losses(h['train_losses'], h['val_losses'])

In [11]:
# Load test data
with open(f'test_data/{benchmark_name}_test.pkl', 'rb') as f:
    test_dict = pickle.load(f)
test_dict = custom_configure(test_dict, preconfigured=True)

In [12]:
# Simulate from surrogate simulator
x_sim_s = amortizer.sample_data(test_dict, n_samples=1)
x_sim_s = tf.squeeze(x_sim_s)

# Sample from posteriors given surrogate outputs
post_samples_s = amortizer.sample_parameters({'direct_conditions': x_sim_s}, n_samples=250)

# Sample from posteriors given simulator outputs
post_samples = amortizer.sample_parameters(test_dict, n_samples=250)

# Prior samples
prior_samples = test_dict['posterior_inputs']['parameters']

In [13]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    'text.latex.preamble' : r'\usepackage{{amsmath}}'
})

## Individual Calibration Plots

In [14]:
# Posterior given surrogate outputs
f = plot_sbc_ecdf(
    post_samples, 
    prior_samples, 
    ecdf_bands_kwargs=dict(confidence=0.95), 
    label_fontsize=24,
    legend_fontsize=16,
    difference=True, 
    param_names=[r'$\theta_1$:\,X Location', r'$\theta_2$:\,Y Location'], 
    rank_ecdf_colors=['#000080', '#9999FF']
)

f.savefig(f'figures/{benchmark_name}_diff_ind_post.pdf', dpi=300, bbox_inches='tight')

In [15]:
# Posterior given surrogate outputs
f = plot_sbc_ecdf(
    post_samples_s,
    prior_samples, 
    ecdf_bands_kwargs=dict(confidence=0.95), 
    label_fontsize=24, 
    legend_fontsize=16,
    difference=True, 
    param_names=[r'$\theta_1$:\,X Location', r'$\theta_2$:\,Y Location'], 
    rank_ecdf_colors=['#800000', '#FF9999']
)

f.savefig(f'figures/{benchmark_name}_diff_ind_joint.pdf', dpi=300, bbox_inches='tight')

# Appendix Plots

In [16]:
# load custom tighter plotting function
from custom_plots import plot_sbc_ecdf_appendix

# adjust for better readability
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "serif",
    'text.latex.preamble' : r'\usepackage{{amsmath}}'
})

### Loss History

In [17]:
# Use loaded history, since reference 'h' will only exist after training
h = trainer.loss_history.get_plottable()
f = plot_losses(h['train_losses'], h['val_losses'])
plt.savefig(f"figures/{benchmark_name}_losses.pdf", dpi=300, bbox_inches="tight")

### Posterior Calibration

In [18]:
f = plot_sbc_ecdf_appendix(
    post_samples, 
    prior_samples, 
    ecdf_bands_kwargs=dict(confidence=0.95), 
    param_names=benchmark.benchmark_info['parameter_names'],
    label_fontsize=24, 
    legend_fontsize=24, 
    difference=True, 
    rank_ecdf_color='#000080'
)

plt.savefig(f"figures/{benchmark_name}_posterior_calibration_diff_separate.pdf", dpi=300, bbox_inches="tight")

### Joint Calibration

In [19]:
f = plot_sbc_ecdf_appendix(
    post_samples_s, 
    prior_samples, 
    ecdf_bands_kwargs=dict(confidence=0.95), 
    param_names=benchmark.benchmark_info['parameter_names'],
    label_fontsize=24, 
    legend_fontsize=24, 
    difference=True, 
    rank_ecdf_color='#800000'
)

plt.savefig(f"figures/{benchmark_name}_joint_calibration_diff_separate.pdf", dpi=300, bbox_inches="tight")